In [14]:
import os
import pandas as pd
from datetime import datetime

In [27]:
df = pd.read_csv(os.getcwd()+'/data/customer.csv')

df.head()

,id,first_name,last_name,email_id,birthdate,street_address,zipcode,city,state,phone_number,...,device_protection,streaming_music,streaming_movies,contract,start_date,end_date,monthly_charges,tenure,total_charges,latest_transaction_date
0,1,Eustace,McCoid,emccoid0@parallels.com,5/18/1959,0308 Bunker Hill Parkway,92844,Garden Grove,California,310-554-8521,...,No,No,No,Annual,10/30/2017,10/30/2018,40,12,480,9/30/2018
1,2,Isador,Maestro,imaestro1@naver.com,5/26/2006,41 Kings Alley,28272,Charlotte,North Carolina,704-996-0008,...,No internet service,No internet service,No internet service,Monthly,5/23/2017,12/31/9999,40,26,1040,5/23/2019
2,3,Irv,Gaskins,igaskins2@gnu.org,1/23/1987,6220 Vermont Lane,24020,Roanoke,Virginia,540-804-0338,...,No,No,Yes,Monthly,10/27/2016,12/31/9999,55,33,2970,7/27/2019
3,4,Bengt,Serot,bserot3@squarespace.com,4/23/1969,67293 Rockefeller Road,70894,Baton Rouge,Louisiana,225-969-8568,...,No internet service,No internet service,No internet service,Annual,7/12/2017,12/31/9999,40,24,960,7/12/2019
4,5,Darrel,Brennon,dbrennon4@ifeng.com,5/1/1993,168 Sage Plaza,29579,Myrtle Beach,South Carolina,843-474-0154,...,No internet service,No internet service,No internet service,Annual,1/8/2017,1/8/2018,40,12,480,12/8/2017


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 34 columns):
id                         10000 non-null int64
first_name                 10000 non-null object
last_name                  10000 non-null object
email_id                   10000 non-null object
birthdate                  10000 non-null object
street_address             10000 non-null object
zipcode                    10000 non-null int64
city                       10000 non-null object
state                      10000 non-null object
phone_number               10000 non-null object
active                     10000 non-null bool
region                     10000 non-null object
gender                     10000 non-null object
age                        10000 non-null int64
connection_type            10000 non-null object
average_call_length        10000 non-null float64
phone_service              10000 non-null object
multiplie_lines            10000 non-null object
paperless_bill

In [29]:
df.describe()

,id,zipcode,age,average_call_length,monthly_charges,tenure,total_charges
count,10000.00000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5000.50000,53110.404900,39.38590,31.288606,47.859300,19.387600,926.279300
std,2886.89568,28925.508964,15.41368,17.272236,17.162708,10.355583,627.372082
min,1.00000,214.000000,13.00000,1.000000,32.000000,5.000000,160.000000
25%,2500.75000,29206.750000,26.00000,16.380000,40.000000,12.000000,480.000000
50%,5000.50000,48915.500000,40.00000,31.540000,40.000000,14.000000,720.000000
75%,7500.25000,79182.000000,53.00000,46.360000,46.000000,28.000000,1218.250000
max,10000.00000,99790.000000,66.00000,61.000000,95.000000,42.000000,3990.000000


In [33]:
# active = df['active'].values
# start_date = df['start_date'].values
# end_date = df['end_date'].values
now = datetime.now()
current_date = now.strftime("%m/%d/%Y")
d1 = datetime.strptime(current_date, "%m/%d/%Y")
recency = []

for index, row in df.iterrows():
    #Calculating recency for active and annual contract rows
    if row['active'] == True:
        lt_date = row['latest_transaction_date']
        d2 = datetime.strptime(lt_date, "%m/%d/%Y")
        recency.append((d1.date()-d2.date()).days)
#     Calculating recency for inactive and annual contract rows
    if row['active'] == False:
        lt_date = row['latest_transaction_date']
        d2 = datetime.strptime(lt_date, "%m/%d/%Y")
        recency.append((d1.date()-d2.date()).days)
    
rfm = pd.DataFrame()
rfm['id'] = df['id']
rfm['frequency'] =  df['tenure']
rfm['monetary'] =  df['total_charges']
rfm['recency'] = recency

rfm.head()

# rfm = df[['id','recency','frequency','total_charges']]

# # rfm = rfm.drop_duplicates()
# rfm.head()

,id,frequency,monetary,recency
0,1,12,480,304
1,2,26,1040,69
2,3,33,2970,4
3,4,24,960,19
4,5,12,480,600


In [34]:
rfm['r_quartile'] = pd.qcut(rfm['recency'], 4, ['1','2','3','4'],duplicates='drop')
rfm['f_quartile'] = pd.qcut(rfm['frequency'], 4, ['4','3','2','1'],duplicates='drop')
rfm['m_quartile'] = pd.qcut(rfm['monetary'], 4, ['4','3','2','1'],duplicates='drop')

rfm['rfm_score'] = rfm.r_quartile.astype(str)+ rfm.f_quartile.astype(str) + rfm.m_quartile.astype(str)

rfm.head()

,id,frequency,monetary,recency,r_quartile,f_quartile,m_quartile,rfm_score
0,1,12,480,304,3,4,4,344
1,2,26,1040,69,2,2,2,222
2,3,33,2970,4,1,1,1,111
3,4,24,960,19,1,2,2,122
4,5,12,480,600,4,4,4,444


In [45]:
rfm['rfm_score'].unique()

array(['344', '222', '111', '122', '444', '312', '233', '442', '422',
       '322', '223', '311', '211', '443', '144', '244', '343', '133',
       '221', '212', '342', '143', '323', '142', '411', '243', '242',
       '232', '121', '132', '321', '421', '134', '112', '131', '224',
       '234', '123', '231', '412'], dtype=object)

### As per the RFM score identifying the segments for every customer
#### Customer with score 111 are best customers
#### Customer with score X1X are loyal customers
#### Customer with score XX1 are big spenders
#### Customer with score 3XX are almost lost customers
#### Customer with score 444 are lost cheap customers
#### Remaining customers are average customers

In [46]:
segment = []

for index, row in rfm.iterrows():
    if row['rfm_score'] == "111":
        segment.append("Best Customer")
    elif row['rfm_score'] in ("311","344","322","343","342","323","321"):
        segment.append("Almost Lost")
    elif row['rfm_score']in ("411","422","443","421","412"):
        segment.append("Lost Customers")
    elif row['rfm_score']== "444":
        segment.append("Lost Cheap Customers")
    elif row['rfm_score'] in ("211","212","112","312"):
        segment.append("Loyal Customers")
    elif row['rfm_score'] in ("211","221","121","231"):
        segment.append("Big Spenders")
    else:
        segment.append("Average Customers")
    
#rfm[rfm['RFM_Score']=='411'].sort_values('monetary', ascending=False).head()

In [47]:
#Assigning every customer with segment
df['segment'] = segment

In [48]:
df.head()

,id,first_name,last_name,email_id,birthdate,street_address,zipcode,city,state,phone_number,...,streaming_music,streaming_movies,contract,start_date,end_date,monthly_charges,tenure,total_charges,latest_transaction_date,segment
0,1,Eustace,McCoid,emccoid0@parallels.com,5/18/1959,0308 Bunker Hill Parkway,92844,Garden Grove,California,310-554-8521,...,No,No,Annual,10/30/2017,10/30/2018,40,12,480,9/30/2018,Almost Lost
1,2,Isador,Maestro,imaestro1@naver.com,5/26/2006,41 Kings Alley,28272,Charlotte,North Carolina,704-996-0008,...,No internet service,No internet service,Monthly,5/23/2017,12/31/9999,40,26,1040,5/23/2019,Average Customers
2,3,Irv,Gaskins,igaskins2@gnu.org,1/23/1987,6220 Vermont Lane,24020,Roanoke,Virginia,540-804-0338,...,No,Yes,Monthly,10/27/2016,12/31/9999,55,33,2970,7/27/2019,Best Customer
3,4,Bengt,Serot,bserot3@squarespace.com,4/23/1969,67293 Rockefeller Road,70894,Baton Rouge,Louisiana,225-969-8568,...,No internet service,No internet service,Annual,7/12/2017,12/31/9999,40,24,960,7/12/2019,Average Customers
4,5,Darrel,Brennon,dbrennon4@ifeng.com,5/1/1993,168 Sage Plaza,29579,Myrtle Beach,South Carolina,843-474-0154,...,No internet service,No internet service,Annual,1/8/2017,1/8/2018,40,12,480,12/8/2017,Lost Cheap Customers


In [50]:
df.to_csv(os.getcwd()+"/data/final.csv",index=False)